In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from functions import runge, MSE, R2, Ridge_parameters, OLS_parameters
from functions import OLS_gradient, Ridge_gradient, Lasso_gradient, polynomial_features
import json

In [ ]:
# Stochastic GD

# Data generation
n = 1000
np.random.seed(42)
x = np.linspace(-1,1, n)
y = runge(x) + 0.1*np.random.normal(0,1)

# Split into training and test sets, scale data and create polynomial features
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = polynomial_features(x_train, 10)
X_test = polynomial_features(x_test, 10)
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
y_offset = np.mean(y_train)

lmbda = 0.001

# Calculate parameters using OLS and Ridge closed form solutions
beta_r = Ridge_parameters(X_train_s, y_train, lmbda)
beta_o = OLS_parameters(X_train_s, y_train)

# Initialize weights for gradient descent
beta_gd_ols = np.zeros(len(beta_r))
beta_gd_ridge = np.zeros(len(beta_r))
beta_gd_lasso = np.zeros(len(beta_r))

# Initialize hyperparameters
n_epochs = 50000
stopping_criteria = [1e-10]*len(beta_r)
M = 50   #size of each minibatch
m = int(n/M) #number of minibatches
t0, t1 = 5, 50
def learning_schedule(t):
    return t0/(t+t1)

# Stochastic Gradient Descent for Lasso Regression
for epoch in range(n_epochs):
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Lasso_gradient(xi, yi, beta_gd_lasso, lmbda)
        eta = learning_schedule(epoch*m+i)
        if np.linalg.norm(eta*gradients) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_lasso = beta_gd_lasso - eta*gradients

# Stochastic Gradient Descent for OLS
for epoch in range(n_epochs):
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = OLS_gradient(xi, yi, beta_gd_ols)
        eta = learning_schedule(epoch*m+i)
        if np.linalg.norm(eta*gradients) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_ols = beta_gd_ols - eta*gradients

# Stochastic Gradient Descent for Ridge Regression
for epoch in range(n_epochs):
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Ridge_gradient(xi, yi, beta_gd_ridge, lmbda)
        eta = learning_schedule(epoch*m+i)
        if np.linalg.norm(eta*gradients) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_ridge = beta_gd_ridge - eta*gradients

y_gd_lasso = X_test_s @ beta_gd_lasso + y_offset
y_gd_ols = X_test_s @ beta_gd_ols + y_offset
y_gd_ridge = X_test_s @ beta_gd_ridge + y_offset

mse_sgd_ols = MSE(y_test, y_gd_ols)
mse_sgd_ridge = MSE(y_test, y_gd_ridge)
mse_sgd_lasso = MSE(y_test, y_gd_lasso)
r2_sgd_ols = R2(y_test, y_gd_ols)
r2_sgd_ridge = R2(y_test, y_gd_ridge)
r2_sgd_lasso = R2(y_test, y_gd_lasso)

print(f"MSE GD Lasso: {mse_sgd_lasso}, MSE GD OLS: {mse_sgd_ols}, MSE GD Ridge: {mse_sgd_ridge}")
print(f"R2 GD Lasso: {r2_sgd_lasso}, R2 GD OLS: {r2_sgd_ols}, R2 GD Ridge: {r2_sgd_ridge}")
print(f"Beta GD Lasso: {beta_gd_lasso}")
print(f"Beta GD OLS: {beta_gd_ols}")
print(f"Beta GD Ridge: {beta_gd_ridge}")    
print("--------------------------------------------------")

dict_sgd = {'MSE GD Lasso': mse_sgd_lasso,
                        'R2 GD Lasso': r2_sgd_lasso,
                        'Beta GD Lasso': beta_gd_lasso,
                        'MSE GD OLS': mse_sgd_ols,
                        'R2 GD OLS': r2_sgd_ols,
                        'Beta GD OLS': beta_gd_ols,
                        'MSE GD Ridge': mse_sgd_ridge,
                        'R2 GD Ridge': r2_sgd_ridge,
                        'Beta GD Ridge': beta_gd_ridge}
with open('sgd_results.json', 'w') as f:
    json.dump(dict_sgd, f, indent=4, default=lambda x: x.tolist() if hasattr(x, 'tolist') else x)

In [ ]:
#Addition of momentum

# Data generation
n = 1000
np.random.seed(42)
x = np.linspace(-1,1, n)
y = runge(x) + 0.1*np.random.normal(0,1)

# Split into training and test sets, scale data and create polynomial features
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = polynomial_features(x_train, 10)
X_test = polynomial_features(x_test, 10)
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
y_offset = np.mean(y_train)

lmbda = 0.001

# Calculate parameters using OLS and Ridge closed form solutions
beta_r = Ridge_parameters(X_train_s, y_train, lmbda)
beta_o = OLS_parameters(X_train_s, y_train)

# Initialize weights for gradient descent
beta_gd_r = np.zeros(len(beta_r))
beta_gd_o = np.zeros(len(beta_o))
beta_gd_lasso = np.zeros(len(beta_r))

# Initialize hyperparameters
num_iters = 100000000
momentum = 0.3
stopping_criteria = [1e-10]*len(beta_r)
change = 0.0
n_epochs = 500
M = 50   #size of each minibatch
m = int(n/M) #number of minibatches
t0, t1 = 5, 50
def learning_schedule(t):
    return t0/(t+t1)

for epoch in range(n_epochs):
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Lasso_gradient(xi, yi, beta_gd_lasso, lmbda)
        eta = learning_schedule(epoch*m+i)
        # calculate update
        new_change = eta*gradients+momentum*change
        if np.linalg.norm(new_change) < np.linalg.norm(stopping_criteria):
            break
        # take a step
        beta_gd_lasso -= new_change
        # save the change
        change = new_change

change = 0.0
# Gradient descent loop
for epoch in range(n_epochs):
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Ridge_gradient(xi, yi, beta_gd_ridge, lmbda)
        eta = learning_schedule(epoch*m+i)
        # calculate update
        new_change = eta*gradients+momentum*change
        if np.linalg.norm(new_change) < np.linalg.norm(stopping_criteria):
            break
        # take a step
        beta_gd_ridge -= new_change
        # save the change
        change = new_change

change = 0.0
for epoch in range(n_epochs):
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = OLS_gradient(xi, yi, beta_gd_ols)
        eta = learning_schedule(epoch*m+i)
        # calculate update
        new_change = eta*gradients+momentum*change
        if np.linalg.norm(new_change) < np.linalg.norm(stopping_criteria):
            break
        # take a step
        beta_gd_ols -= new_change
        # save the change
        change = new_change

y_gd_lasso = X_test_s @ beta_gd_lasso + y_offset
y_gd_ols = X_test_s @ beta_gd_ols + y_offset
y_gd_ridge = X_test_s @ beta_gd_ridge + y_offset

mse_sgd_ols = MSE(y_test, y_gd_ols)
mse_sgd_ridge = MSE(y_test, y_gd_ridge)
mse_sgd_lasso = MSE(y_test, y_gd_lasso)
r2_sgd_ols = R2(y_test, y_gd_ols)
r2_sgd_ridge = R2(y_test, y_gd_ridge)
r2_sgd_lasso = R2(y_test, y_gd_lasso)

print(f"MSE GD Lasso: {mse_sgd_lasso}, MSE GD OLS: {mse_sgd_ols}, MSE GD Ridge: {mse_sgd_ridge}")
print(f"R2 GD Lasso: {r2_sgd_lasso}, R2 GD OLS: {r2_sgd_ols}, R2 GD Ridge: {r2_sgd_ridge}")
print(f"Beta GD Lasso: {beta_gd_lasso}")
print(f"Beta GD OLS: {beta_gd_ols}")
print(f"Beta GD Ridge: {beta_gd_ridge}")    
print("--------------------------------------------------")

dict_sgd_momentum = {'MSE GD Lasso': mse_sgd_lasso,
                        'R2 GD Lasso': r2_sgd_lasso,
                        'Beta GD Lasso': beta_gd_lasso,
                        'MSE GD OLS': mse_sgd_ols,
                        'R2 GD OLS': r2_sgd_ols,
                        'Beta GD OLS': beta_gd_ols,
                        'MSE GD Ridge': mse_sgd_ridge,
                        'R2 GD Ridge': r2_sgd_ridge,
                        'Beta GD Ridge': beta_gd_ridge}
with open('sgd_momentum_results.json', 'w') as f:
    json.dump(dict_sgd_momentum, f, indent=4, default=lambda x: x.tolist() if hasattr(x, 'tolist') else x)

Ridge parameters: [ 0.00000000e+00  5.54300134e-03 -2.59074814e+00 -7.44146292e-03
  9.69505866e+00 -9.50339784e-03 -1.68243883e+01  2.65540645e-02
  1.37520890e+01 -1.42341245e-02 -4.27508408e+00]
OLS parameters: [ 0.00000000e+00  3.74356233e-03 -2.97426630e+00  9.69057072e-03
  1.25136327e+01 -7.10156122e-02 -2.40999002e+01  1.12354140e-01
  2.15580712e+01 -5.44668224e-02 -7.24703004e+00]
Convergence reached at iteration for Ridge 22131
Convergence reached at iteration for OLS 8405293
Learning rate: 0.2
MSE OLS: 0.001484546631866829, MSE Ridge: 0.0017395603039358032, MSE GD OLS: 0.0014845457008599405, MSE GD Ridge: 0.009755924145600206
R2 OLS: 0.9800049681741101, R2 Ridge: 0.976567517228484, R2 GD OLS: 0.9800049806763442, R2 GD Ridge: 0.8688669119993857
Beta GD OLS: [ 0.00000000e+00  3.74359499e-03 -2.97426162e+00  9.69026586e-03
  1.25135981e+01 -7.10145701e-02 -2.40998104e+01  1.12352739e-01
  2.15579745e+01 -5.44661837e-02 -7.24699315e+00], Beta GD Ridge: [ 0.00000000e+00  1.91891

In [ ]:
#ADAgrad

# Data generation
n = 1000
np.random.seed(42)
x = np.linspace(-1,1, n)
y = runge(x) + 0.1*np.random.normal(0,1)

# Split into training and test sets, scale data and create polynomial features
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = polynomial_features(x_train, 10)
X_test = polynomial_features(x_test, 10)
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
y_offset = np.mean(y_train)

lmbda = 0.001

# Calculate parameters using OLS and Ridge closed form solutions
beta_r = Ridge_parameters(X_train_s, y_train, lmbda)
beta_o = OLS_parameters(X_train_s, y_train)

# Initialize weights for gradient descent
beta_gd_r = np.zeros(len(beta_r))
beta_gd_o = np.zeros(len(beta_o))
beta_gd_lasso = np.zeros(len(beta_r))

# Hessian matrix for Ridge
H = (2.0/n)* X_train_s.T @ X_train_s + 2*lmbda*np.eye(X_train_s.shape[1])
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix:{EigValues}")

# Initialize hyperparameters
lr = 1.0 / np.max(EigValues)
num_iters = 100000000
stopping_criteria = [1e-10]*len(beta_r)
delta = 1e-8
n_epochs = 500
M = 50   #size of each minibatch
m = int(n/M) #number of minibatches

for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Ridge_gradient(xi, yi, beta_gd_ridge, lmbda)
        Giter += gradients*gradients
        update = gradients*lr/(delta+np.sqrt(Giter))
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_ridge -= update

# Hessian matrix for Ridge
H = (2.0/n)* X_train_s.T @ X_train_s
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix:{EigValues}")

# Initialize hyperparameters
lr = 1.0 / np.max(EigValues)

for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Lasso_gradient(xi, yi, beta_gd_lasso, lmbda)
        Giter += gradients*gradients
        update = gradients*lr/(delta+np.sqrt(Giter))
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_lasso -= update

for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = OLS_gradient(xi, yi, beta_gd_ols)
        Giter += gradients*gradients
        update = gradients*lr/(delta+np.sqrt(Giter))
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_ols -= update

y_gd_lasso = X_test_s @ beta_gd_lasso + y_offset
y_gd_ols = X_test_s @ beta_gd_ols + y_offset
y_gd_ridge = X_test_s @ beta_gd_ridge + y_offset

mse_sgd_ols = MSE(y_test, y_gd_ols)
mse_sgd_ridge = MSE(y_test, y_gd_ridge)
mse_sgd_lasso = MSE(y_test, y_gd_lasso)
r2_sgd_ols = R2(y_test, y_gd_ols)
r2_sgd_ridge = R2(y_test, y_gd_ridge)
r2_sgd_lasso = R2(y_test, y_gd_lasso)

print(f"MSE GD Lasso: {mse_sgd_lasso}, MSE GD OLS: {mse_sgd_ols}, MSE GD Ridge: {mse_sgd_ridge}")
print(f"R2 GD Lasso: {r2_sgd_lasso}, R2 GD OLS: {r2_sgd_ols}, R2 GD Ridge: {r2_sgd_ridge}")
print(f"Beta GD Lasso: {beta_gd_lasso}")
print(f"Beta GD OLS: {beta_gd_ols}")
print(f"Beta GD Ridge: {beta_gd_ridge}")    
print("--------------------------------------------------")

dict_sgd_adagrad = {'MSE GD Lasso': mse_sgd_lasso,
                        'R2 GD Lasso': r2_sgd_lasso,
                        'Beta GD Lasso': beta_gd_lasso,
                        'MSE GD OLS': mse_sgd_ols,
                        'R2 GD OLS': r2_sgd_ols,
                        'Beta GD OLS': beta_gd_ols,
                        'MSE GD Ridge': mse_sgd_ridge,
                        'R2 GD Ridge': r2_sgd_ridge,
                        'Beta GD Ridge': beta_gd_ridge}
with open('sgd_adagrad_results.json', 'w') as f:
    json.dump(dict_sgd_adagrad, f, indent=4, default=lambda x: x.tolist() if hasattr(x, 'tolist') else x)

Ridge parameters: [ 0.00000000e+00  5.54300134e-03 -2.59074814e+00 -7.44146292e-03
  9.69505866e+00 -9.50339784e-03 -1.68243883e+01  2.65540645e-02
  1.37520890e+01 -1.42341245e-02 -4.27508408e+00]
OLS parameters: [ 0.00000000e+00  3.74356233e-03 -2.97426630e+00  9.69057072e-03
  1.25136327e+01 -7.10156122e-02 -2.40999002e+01  1.12354140e-01
  2.15580712e+01 -5.44668224e-02 -7.24703004e+00]
Eigenvalues of Hessian Matrix:[7.81988102e+00 7.04651821e+00 6.45820952e-01 4.34618809e-01
 4.34213818e-02 1.99862028e-02 3.31440644e-03 2.41897568e-03
 2.00413804e-03 2.01589893e-03 2.00000000e-03]
Convergence reached at iteration for Ridge 50479
Eigenvalues of Hessian Matrix:[7.81788102e+00 7.04451821e+00 6.43820952e-01 4.32618809e-01
 4.14213818e-02 1.79862028e-02 1.31440644e-03 4.18975683e-04
 4.13804044e-06 1.58989315e-05 0.00000000e+00]


In [ ]:
#RMSprop

# Data generation
n = 1000
np.random.seed(42)
x = np.linspace(-1,1, n)
y = runge(x) + 0.1*np.random.normal(0,1)

# Split into training and test sets, scale data and create polynomial features
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = polynomial_features(x_train, 10)
X_test = polynomial_features(x_test, 10)
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
y_offset = np.mean(y_train)

lmbda = 0.001

# Calculate parameters using OLS and Ridge closed form solutions
beta_r = Ridge_parameters(X_train_s, y_train, lmbda)
beta_o = OLS_parameters(X_train_s, y_train)

# Initialize weights for gradient descent
beta_gd_r = np.zeros(len(beta_r))
beta_gd_o = np.zeros(len(beta_o))
beta_gd_lasso = np.zeros(len(beta_r))

# Hessian matrix for Ridge
H = (2.0/n)* X_train_s.T @ X_train_s + 2*lmbda*np.eye(X_train_s.shape[1])
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix:{EigValues}")

# Initialize hyperparameters
lr = 1.0 / np.max(EigValues)
num_iters = 100000000
stopping_criteria = [1e-10]*len(beta_r)
delta = 1e-8
rho = 0.99
n_epochs = 500
M = 50   #size of each minibatch
m = int(n/M) #number of minibatches

for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Ridge_gradient(xi, yi, beta_gd_ridge, lmbda)
	# Accumulated gradient
	# Scaling with rho the new and the previous results
        Giter = (rho*Giter+(1-rho)*gradients*gradients)
	# Taking the diagonal only and inverting
        update = gradients*lr/(delta+np.sqrt(Giter))
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
	# Hadamard product
        beta_gd_ridge -= update

# Hessian matrix for OLS
H = (2.0/n)* X_train_s.T @ X_train_s 
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix:{EigValues}")

# Initialize hyperparameters
lr = 1.0 / np.max(EigValues)

for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Lasso_gradient(xi, yi, beta_gd_lasso, lmbda)
	# Accumulated gradient
	# Scaling with rho the new and the previous results
        Giter = (rho*Giter+(1-rho)*gradients*gradients)
	# Taking the diagonal only and inverting
        update = gradients*lr/(delta+np.sqrt(Giter))
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
	# Hadamard product
        beta_gd_lasso -= update

G_iter = 0.0

for epoch in range(n_epochs):
    Giter = 0.0
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = OLS_gradient(xi, yi, beta_gd_ols)
	# Accumulated gradient
	# Scaling with rho the new and the previous results
        Giter = (rho*Giter+(1-rho)*gradients*gradients)
	# Taking the diagonal only and inverting
        update = gradients*lr/(delta+np.sqrt(Giter))
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
	# Hadamard product
        beta_gd_ols -= update

y_gd_lasso = X_test_s @ beta_gd_lasso + y_offset
y_gd_ols = X_test_s @ beta_gd_ols + y_offset
y_gd_ridge = X_test_s @ beta_gd_ridge + y_offset

mse_sgd_ols = MSE(y_test, y_gd_ols)
mse_sgd_ridge = MSE(y_test, y_gd_ridge)
mse_sgd_lasso = MSE(y_test, y_gd_lasso)
r2_sgd_ols = R2(y_test, y_gd_ols)
r2_sgd_ridge = R2(y_test, y_gd_ridge)
r2_sgd_lasso = R2(y_test, y_gd_lasso)

print(f"MSE GD Lasso: {mse_sgd_lasso}, MSE GD OLS: {mse_sgd_ols}, MSE GD Ridge: {mse_sgd_ridge}")
print(f"R2 GD Lasso: {r2_sgd_lasso}, R2 GD OLS: {r2_sgd_ols}, R2 GD Ridge: {r2_sgd_ridge}")
print(f"Beta GD Lasso: {beta_gd_lasso}")
print(f"Beta GD OLS: {beta_gd_ols}")
print(f"Beta GD Ridge: {beta_gd_ridge}")    
print("--------------------------------------------------")

dict_sgd_rmsprop = {'MSE GD Lasso': mse_sgd_lasso,
                        'R2 GD Lasso': r2_sgd_lasso,
                        'Beta GD Lasso': beta_gd_lasso,
                        'MSE GD OLS': mse_sgd_ols,
                        'R2 GD OLS': r2_sgd_ols,
                        'Beta GD OLS': beta_gd_ols,
                        'MSE GD Ridge': mse_sgd_ridge,
                        'R2 GD Ridge': r2_sgd_ridge,
                        'Beta GD Ridge': beta_gd_ridge}
with open('sgd_rmsprop_results.json', 'w') as f:
    json.dump(dict_sgd_rmsprop, f, indent=4, default=lambda x: x.tolist() if hasattr(x, 'tolist') else x)

Ridge parameters: [ 0.00000000e+00  5.54300134e-03 -2.59074814e+00 -7.44146292e-03
  9.69505866e+00 -9.50339784e-03 -1.68243883e+01  2.65540645e-02
  1.37520890e+01 -1.42341245e-02 -4.27508408e+00]
OLS parameters: [ 0.00000000e+00  3.74356233e-03 -2.97426630e+00  9.69057072e-03
  1.25136327e+01 -7.10156122e-02 -2.40999002e+01  1.12354140e-01
  2.15580712e+01 -5.44668224e-02 -7.24703004e+00]
Eigenvalues of Hessian Matrix:[7.81988102e+00 7.04651821e+00 6.45820952e-01 4.34618809e-01
 4.34213818e-02 1.99862028e-02 3.31440644e-03 2.41897568e-03
 2.00413804e-03 2.01589893e-03 2.00000000e-03]


In [ ]:
# ADAM

# Data generation
n = 1000
np.random.seed(42)
x = np.linspace(-1,1, n)
y = runge(x) + 0.1*np.random.normal(0,1)

# Split into training and test sets, scale data and create polynomial features
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = polynomial_features(x_train, 10)
X_test = polynomial_features(x_test, 10)
scaler.fit(X_train)
X_train_s = scaler.transform(X_train)
X_test_s = scaler.transform(X_test)
y_offset = np.mean(y_train)

lmbda = 0.001

# Calculate parameters using OLS and Ridge closed form solutions
beta_r = Ridge_parameters(X_train_s, y_train, lmbda)
beta_o = OLS_parameters(X_train_s, y_train)

# Initialize weights for gradient descent
beta_gd_r = np.zeros(len(beta_r))
beta_gd_o = np.zeros(len(beta_o))
beta_gd_lasso = np.zeros(len(beta_r))

# Hessian matrix for Ridge
H = (2.0/n)* X_train_s.T @ X_train_s + 2*lmbda*np.eye(X_train_s.shape[1])
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix:{EigValues}")

# Initialize hyperparameters
lr = 1.0 / np.max(EigValues)
num_iters = 100000000
stopping_criteria = [1e-10]*len(beta_r)
delta = 1e-8
rho_1 = 0.9
rho_2 = 0.999
n_epochs = 500
M = 50   #size of each minibatch
m = int(n/M) #number of minibatches
iter = 0

for epoch in range(n_epochs):
    first_moment = 0.0
    second_moment = 0.0
    iter += 1
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Ridge_gradient(xi, yi, beta_gd_ridge, lmbda)
        # Computing moments first
        first_moment = rho_1*first_moment + (1-rho_1)*gradients
        second_moment = rho_2*second_moment+(1-rho_2)*gradients*gradients
        first_term = first_moment/(1.0-rho_1**iter)
        second_term = second_moment/(1.0-rho_2**iter)
	# Scaling with rho the new and the previous results
        update = lr*first_term/(np.sqrt(second_term)+delta)
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_ridge -= update

# Hessian matrix for OLS
H = (2.0/n)* X_train_s.T @ X_train_s 
EigValues, EigVectors = np.linalg.eig(H)
print(f"Eigenvalues of Hessian Matrix:{EigValues}")

# Initialize hyperparameters
lr = 1.0 / np.max(EigValues)
iter = 0

for epoch in range(n_epochs):
    first_moment = 0.0
    second_moment = 0.0
    iter += 1
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = Lasso_gradient(xi, yi, beta_gd_lasso, lmbda)
        # Computing moments first
        first_moment = rho_1*first_moment + (1-rho_1)*gradients
        second_moment = rho_2*second_moment+(1-rho_2)*gradients*gradients
        first_term = first_moment/(1.0-rho_1**iter)
        second_term = second_moment/(1.0-rho_2**iter)
	# Scaling with rho the new and the previous results
        update = lr*first_term/(np.sqrt(second_term)+delta)
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_lasso -= update

iter = 0

for epoch in range(n_epochs):
    first_moment = 0.0
    second_moment = 0.0
    iter += 1
    for i in range(m):
        random_index = M*np.random.randint(m)
        xi = X_train_s[random_index:random_index+M]
        yi = y_train[random_index:random_index+M]
        gradients = OLS_gradient(xi, yi, beta_gd_ols)
        # Computing moments first
        first_moment = rho_1*first_moment + (1-rho_1)*gradients
        second_moment = rho_2*second_moment+(1-rho_2)*gradients*gradients
        first_term = first_moment/(1.0-rho_1**iter)
        second_term = second_moment/(1.0-rho_2**iter)
	# Scaling with rho the new and the previous results
        update = lr*first_term/(np.sqrt(second_term)+delta)
        if np.linalg.norm(update) < np.linalg.norm(stopping_criteria):
            break
        beta_gd_ols -= update

y_gd_lasso = X_test_s @ beta_gd_lasso + y_offset
y_gd_ols = X_test_s @ beta_gd_ols + y_offset
y_gd_ridge = X_test_s @ beta_gd_ridge + y_offset

mse_sgd_ols = MSE(y_test, y_gd_ols)
mse_sgd_ridge = MSE(y_test, y_gd_ridge)
mse_sgd_lasso = MSE(y_test, y_gd_lasso)
r2_sgd_ols = R2(y_test, y_gd_ols)
r2_sgd_ridge = R2(y_test, y_gd_ridge)
r2_sgd_lasso = R2(y_test, y_gd_lasso)

print(f"MSE GD Lasso: {mse_sgd_lasso}, MSE GD OLS: {mse_sgd_ols}, MSE GD Ridge: {mse_sgd_ridge}")
print(f"R2 GD Lasso: {r2_sgd_lasso}, R2 GD OLS: {r2_sgd_ols}, R2 GD Ridge: {r2_sgd_ridge}")
print(f"Beta GD Lasso: {beta_gd_lasso}")
print(f"Beta GD OLS: {beta_gd_ols}")
print(f"Beta GD Ridge: {beta_gd_ridge}")    
print("--------------------------------------------------")

dict_sgd_adam = {'MSE GD Lasso': mse_sgd_lasso,
                        'R2 GD Lasso': r2_sgd_lasso,
                        'Beta GD Lasso': beta_gd_lasso,
                        'MSE GD OLS': mse_sgd_ols,
                        'R2 GD OLS': r2_sgd_ols,
                        'Beta GD OLS': beta_gd_ols,
                        'MSE GD Ridge': mse_sgd_ridge,
                        'R2 GD Ridge': r2_sgd_ridge,
                        'Beta GD Ridge': beta_gd_ridge}
with open('sgd_adam_results.json', 'w') as f:
    json.dump(dict_sgd_adam, f, indent=4, default=lambda x: x.tolist() if hasattr(x, 'tolist') else x)

Ridge parameters: [ 0.00000000e+00  5.54300134e-03 -2.59074814e+00 -7.44146292e-03
  9.69505866e+00 -9.50339784e-03 -1.68243883e+01  2.65540645e-02
  1.37520890e+01 -1.42341245e-02 -4.27508408e+00]
OLS parameters: [ 0.00000000e+00  3.74356233e-03 -2.97426630e+00  9.69057072e-03
  1.25136327e+01 -7.10156122e-02 -2.40999002e+01  1.12354140e-01
  2.15580712e+01 -5.44668224e-02 -7.24703004e+00]
Eigenvalues of Hessian Matrix:[7.81988102e+00 7.04651821e+00 6.45820952e-01 4.34618809e-01
 4.34213818e-02 1.99862028e-02 3.31440644e-03 2.41897568e-03
 2.00413804e-03 2.01589893e-03 2.00000000e-03]
Convergence reached at iteration for Ridge 2060
Eigenvalues of Hessian Matrix:[7.81788102e+00 7.04451821e+00 6.43820952e-01 4.32618809e-01
 4.14213818e-02 1.79862028e-02 1.31440644e-03 4.18975683e-04
 4.13804044e-06 1.58989315e-05 0.00000000e+00]
Convergence reached at iteration for OLS 660108
Learning rate: 0.12791189798390853
MSE OLS: 0.10997884456748792, MSE Ridge: 0.10913833858639004, MSE GD OLS: 0.0